# Prepare dataset for training and validation

In [1]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16
image_height = 480
image_width = 640

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../images/ciffer_julian/train',  # this is the target directory
        target_size=(image_height, image_width),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../images/ciffer_julian/validation',
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')

Using TensorFlow backend.


Found 168 images belonging to 6 classes.
Found 57 images belonging to 6 classes.


# Create the model

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_height, image_width, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 478, 638, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 478, 638, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 239, 319, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 237, 317, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 237, 317, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 118, 158, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 116, 156, 32)      9248      
__________

# Train the network

In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1600 // batch_size,
        epochs=2,
        validation_data=validation_generator,
        validation_steps=16  // batch_size)

Epoch 1/2
100/100 [==============================] - 643s - loss: 1.8304 - acc: 0.1950 - val_loss: 1.6346 - val_acc: 0.5625

In [ ]:
model.save_weights('../models/ciffer_julian.h5')

# Test the network with unknown data

In [ ]:
test_generator = test_datagen.flow_from_directory(
        '../images/ciffer_julian/test',
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model.evaluate_generator(
        test_generator,
        steps=16 // batch_size)